# Assignment Part A

Student Name: Kuah Jia Chen <br>
Student ID: 32286988

## Task 1. Mongo Data Model

1. Based on the two data sets provided i.e. hotspot_historic.csv and climate_historic.csv, design a suitable data model to support the efficient querying of the two data sets in MongoDB. Justify your data model design. The output of this task should be
- An example of the data model.
- The justification for choosing that data model.

### Answer

I will be choosing to construct an embedded data model for this assignment.

(1) An example of the data model.

`The code I used to generate the example of the data model:`
```
from datetime import datetime

target_date = '27-12-2021'
target_date_obj = datetime.strptime(target_date, '%d-%m-%Y')

cursor = db.embedded_climate_hotspot.find(
    {'date':target_date_obj}
)
for document in cursor: 
    document['date'] = document['date'].strftime('%d-%m-%Y') # make the date more readable
    pprint(document)
```

`The example of the data model:`
```
{'GHI_w/m2': 223,
 '_id': ObjectId('6283030ad9df33cead533f80'),
 'air_temperature_celcius': 28,
 'date': '27-12-2021',
 'hotspots': [{'_id': ObjectId('6283030dd9df33cead533f86'),
               'confidence': 78,
               'date': datetime.datetime(2021, 12, 27, 0, 0),
               'datetime': datetime.datetime(2021, 12, 27, 4, 16, 51),
               'latitude': -37.966,
               'longitude': 145.05100000000002,
               'surface_temperature_celcius': 68},
              {'_id': ObjectId('6283030dd9df33cead533f87'),
               'confidence': 82,
               'date': datetime.datetime(2021, 12, 27, 0, 0),
               'datetime': datetime.datetime(2021, 12, 27, 0, 2, 15),
               'latitude': -35.541,
               'longitude': 143.311,
               'surface_temperature_celcius': 63},
              {'_id': ObjectId('6283030dd9df33cead533f88'),
               'confidence': 67,
               'date': datetime.datetime(2021, 12, 27, 0, 0),
               'datetime': datetime.datetime(2021, 12, 27, 0, 2, 15),
               'latitude': -35.554,
               'longitude': 143.30700000000002,
               'surface_temperature_celcius': 53},
              {'_id': ObjectId('6283030dd9df33cead533f89'),
               'confidence': 86,
               'date': datetime.datetime(2021, 12, 27, 0, 0),
               'datetime': datetime.datetime(2021, 12, 27, 0, 2, 14),
               'latitude': -35.543,
               'longitude': 143.316,
               'surface_temperature_celcius': 67}],
 'max_wind_speed': 15.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 58.3,
 'station': 948702,
 'windspeed_knots': 9.3}
```
`Explanation of how the data is stored:`

The example above shows one of the documents in my result collection using the embedded data model. To construct the embedded data model, for each climate data, I will create an array called "hotspots" and for each hotspot data with the same date as this climate data, I will append them to this array. The example above is the climate data for 27/12/2021. We can see that in the "hotspots" array, it stores all the hotspot data that happens on 27/12/2021. 


(2) The justification for choosing that data model.
- According to the questions in task 2, I am required to obtain data from both CSV files. Hence, if I use referencing data model, I have to use aggregation to perform the left outer join several times for the queries in task 2, and thus the total join cost will be expensive and time-consuming when the data becomes big. Besides that, using the referencing data model and doing the join at the programming level can be very complex. However, for the embedded data model, I do not have to perform any join operation as everything is in one collection, therefore this method is fast and easy.

- If I choose the referencing data model, it means that I have two options based on the slides in lecture 7. Assuming I have created two collections named "climate" and "hotspot" based on the data in "climate_historic" and hotspot_historic" respectively. The first method is that for each document of "climate", I will use an array to store all the id of the fire data (i.e., from hotspot) that happens on that date. The second method is that for each document in "hotspot" collection, I will need to create a new field called "climate_id" and it will store the object id of a particular climate document that has the same date as this document. However, for the first method, it would be a problem if the total document of "hotspot" on a particular date is large. The reason is that the array of this document in "climate" will be very long, hence it makes the querying of data inefficient. Besides that, for the second method, if we want to find the data for a particular date, we need to go through every document in a "hotspot", and this has to be done in every query of data. Thus, it is inefficient. Nevertheless, for the embedded data model, since our data is not large, if we want to find the data for a particular date, we could just go to that document and query it, which is more efficient than the previous method. Consequently, I decided to use an embedded data model. 

- Using an embedded data model means everything I need is in a single collection, as a result, it means fewer queries and updates (if necessary) are needed. Thus, it is more convenient. Moreover, since MongoDB does not allow normalization, hence it is fine to have duplicate data or data with a different format in the result collection, so the embedded model is allowed in this situation. 

## Task 2. Querying MongoDB using PyMongo

### 1) Write a python program that will read the data from hotspot_historic.csv and climate_historic.csv and load them to the new database (e.g.fit3182_assignment_db). The collection(s) in fit3182_assignment_db will be based on the document model you have designed in Task A1.
- Please use a csv library to read the files.
- Please DO NOT use the mongo aggregation query to do this task.

In [1]:
from pymongo import MongoClient
from pprint import pprint

# Create a MongoClient/Mongodb connection 
client = MongoClient()

# We will use the database: fit3182_assignment_db
db = client.fit3182_assignment_db

# The collection name is climate and hotspot
climate = db.climate
hotspot = db.hotspot

In [2]:
# delete this collection if it is exists in the database
db.embedded_climate_hotspot.drop()
db.climate.drop()
db.hotspot.drop()

In [3]:
import pandas as pd

# Use csv to read the data
climate_data = pd.read_csv("climate_historic.csv")
climate_data.dtypes
climate_data['date'] = pd.to_datetime(climate_data['date'], format="%d/%m/%y")
climate_data['date'] = climate_data['date'].dt.strftime('%d-%m-%Y')
climate.insert_many(climate_data.to_dict('records'))

In [4]:
# change the data type of date column to date
db.climate.update_many(
   {
      "date":{
         "$type":"string"
      }
   },
   [
      {
         "$set":{
            "date":{
               "$dateFromString":{
                  "dateString":"$date",
                  "format":"%d-%m-%Y"
               }
            }
         }
      }
   ]
)

In [5]:
# Use csv to read the data
hotspot_data = pd.read_csv("hotspot_historic.csv")
hotspot_data['date'] = pd.to_datetime(hotspot_data['date'], format="%d/%m/%Y")
hotspot_data['date'] = hotspot_data['date'].dt.strftime('%d-%m-%Y')
hotspot.insert_many(hotspot_data.to_dict('records'))

In [6]:
# change the data type of date column to date
db.hotspot.update_many(
   {
      "date":{
         "$type":"string"
      }
   },
   [
      {
         "$set":{
            "date":{
               "$dateFromString":{
                  "dateString":"$date",
                  "format":"%d-%m-%Y"
               }
            }
         }
      }
   ]
)

In [7]:
# change the data type of datetime column to datetime
db.hotspot.update_many(
   {
      "datetime":{
         "$type":"string"
      }
   },
   [
      {
         "$set":{
            "datetime":{
               "$dateFromString":{
                  "dateString":"$datetime"
               }
            }
         }
      }
   ]
)

In [8]:
# construct the embedded model
# Prepare the data following the data model with proper data format
all_date = db.climate.distinct("date")                                     # find all the unique date 
for date in all_date:                                                      # for each unique data
    matched_documents = hotspot.find({"date":date})                        # find the hotspot data with the same data 
    current = []                                                           
    for document in matched_documents:
        current.append(document)                                           # append those hotspot data into a single list
    climate.update_one({"date":date},{"$set":{"hotspots":current}})        # add the result list to the climate data

In [9]:
# rename the collection
db.climate.rename("embedded_climate_hotspot")

{'ok': 1.0}

In [10]:
hotspot.drop()

Now, all the data has been successfully loaded into Mongodb. (i.e., the collection in fit3182_assignment_db database)

### 2) Write queries to answer the following tasks on fit3182_assignment_db and corresponding collection(s). You need to write the queries as a python program using the pymongo library in Jupyter Notebook.

#### a) Find climate data on 12th December 2021.

In [11]:
from datetime import datetime

target_date = '12-12-2021'
target_date_obj = datetime.strptime(target_date, '%d-%m-%Y')  # convert the string to date object

cursor = db.embedded_climate_hotspot.find(
    {'date':target_date_obj},
    {"_id":0,"hotspots":0} # hotspots are not needed
)
for document in cursor: 
    document['date'] = document['date'].strftime('%d-%m-%Y') # make the date more readable
    pprint(document)

{'GHI_w/m2': 156,
 'air_temperature_celcius': 19,
 'date': '12-12-2021',
 'max_wind_speed': 12.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


#### b) Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C.

In [12]:
cursor = db.embedded_climate_hotspot.aggregate([
    # since necessary details are in an associative array, hence I need to use unwind
    {"$unwind":"$hotspots"},          
    # make sure the surface temperature was between 65 and 100 inclusive
    {"$match": {"$and":[{"hotspots.surface_temperature_celcius":{"$gt":64}},\
                        {"hotspots.surface_temperature_celcius":{"$lt":101}}]}},
    # only output the necessary details
    {"$project":{"hotspots.latitude":1,"hotspots.longitude":1,"hotspots.surface_temperature_celcius":1,\
                 "hotspots.confidence":1,"_id":0}}
])


for document in cursor: 
    pprint(document['hotspots']) # only print the value in 'hotspots' as that is the necessary part 

{'confidence': 94,
 'latitude': -37.2284,
 'longitude': 147.9187,
 'surface_temperature_celcius': 73}
{'confidence': 97,
 'latitude': -37.6572,
 'longitude': 142.0703,
 'surface_temperature_celcius': 80}
{'confidence': 84,
 'latitude': -37.0193,
 'longitude': 148.1459,
 'surface_temperature_celcius': 71}
{'confidence': 100,
 'latitude': -37.4229,
 'longitude': 147.02700000000002,
 'surface_temperature_celcius': 99}
{'confidence': 80,
 'latitude': -37.0055,
 'longitude': 148.1582,
 'surface_temperature_celcius': 68}
{'confidence': 85,
 'latitude': -37.4128,
 'longitude': 147.0242,
 'surface_temperature_celcius': 98}
{'confidence': 90,
 'latitude': -34.357,
 'longitude': 141.5361,
 'surface_temperature_celcius': 67}
{'confidence': 93,
 'latitude': -34.3539,
 'longitude': 141.5547,
 'surface_temperature_celcius': 72}
{'confidence': 90,
 'latitude': -36.9939,
 'longitude': 148.2244,
 'surface_temperature_celcius': 68}
{'confidence': 95,
 'latitude': -36.9959,
 'longitude': 148.2118,
 'surf

 'latitude': -36.5541,
 'longitude': 144.1321,
 'surface_temperature_celcius': 69}
{'confidence': 91,
 'latitude': -35.6819,
 'longitude': 143.5275,
 'surface_temperature_celcius': 69}
{'confidence': 90,
 'latitude': -35.6871,
 'longitude': 143.5235,
 'surface_temperature_celcius': 68}
{'confidence': 100,
 'latitude': -36.1506,
 'longitude': 143.5671,
 'surface_temperature_celcius': 95}
{'confidence': 93,
 'latitude': -37.4984,
 'longitude': 142.9739,
 'surface_temperature_celcius': 73}
{'confidence': 96,
 'latitude': -37.253,
 'longitude': 144.3681,
 'surface_temperature_celcius': 78}
{'confidence': 92,
 'latitude': -36.9396,
 'longitude': 143.1462,
 'surface_temperature_celcius': 70}
{'confidence': 96,
 'latitude': -36.3387,
 'longitude': 141.7542,
 'surface_temperature_celcius': 77}
{'confidence': 100,
 'latitude': -36.3433,
 'longitude': 143.4369,
 'surface_temperature_celcius': 94}
{'confidence': 100,
 'latitude': -36.345,
 'longitude': 143.4256,
 'surface_temperature_celcius': 87

#### c) Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2021

In [13]:
target_date_1 = '15-12-2021'
target_date_obj_1 = datetime.strptime(target_date_1, '%d-%m-%Y') # convert the string to date object

target_date_2 = '16-12-2021'
target_date_obj_2 = datetime.strptime(target_date_2, '%d-%m-%Y') # convert the string to date object

cursor = db.embedded_climate_hotspot.find(
    { "$or": [ { "date": target_date_obj_1 }, { "date": target_date_obj_2 } ] },
    # only output the necessary details
    {"date":1,"hotspots.surface_temperature_celcius":1,"air_temperature_celcius":1,\
     "relative_humidity":1,"max_wind_speed":1,"_id":0}
)

"""
How to interpret my output:
The "hotspots" array contains all the surface temperature values for hotspots on 15th and 16th Decemeber 2021.
"""

for document in cursor: 
    document['date'] = document['date'].strftime('%d-%m-%Y') # make the date more readable
    pprint(document)

{'air_temperature_celcius': 18,
 'date': '15-12-2021',
 'hotspots': [{'surface_temperature_celcius': 42},
              {'surface_temperature_celcius': 36},
              {'surface_temperature_celcius': 38},
              {'surface_temperature_celcius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': '16-12-2021',
 'hotspots': [{'surface_temperature_celcius': 43},
              {'surface_temperature_celcius': 33},
              {'surface_temperature_celcius': 54},
              {'surface_temperature_celcius': 73},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 75},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 66},
              {'surface_temperature_celcius': 56},
              {'surface_temperature_celcius': 60},
              {'surface_temperature_celcius': 73},
              {'surface_temperature_celcius': 48},
              {'surf

#### d) Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.

In [14]:
cursor = db.embedded_climate_hotspot.aggregate([
    # since necessary details are in an associative array, hence I need to use unwind
    {"$unwind":"$hotspots"},
    # make sure the confidence was between 80 and 100 inclusive
    {"$match": {"$and":[{"hotspots.confidence":{"$gt":79}},\
                        {"hotspots.confidence":{"$lt":101}}]}},
    # only output the necessary details
    {"$project":{"hotspots.datetime":1,"air_temperature_celcius":1,"hotspots.surface_temperature_celcius":1,\
                 "hotspots.confidence":1,"_id":0}}
])

for document in cursor:
    # make the datetime more readable
    document['hotspots']['datetime'] = document['hotspots']['datetime'].strftime("%d-%m-%Y, %H:%M:%S")
    pprint(document) 

{'air_temperature_celcius': 20,
 'hotspots': {'confidence': 87,
              'datetime': '06-03-2021, 05:06:30',
              'surface_temperature_celcius': 62}}
{'air_temperature_celcius': 20,
 'hotspots': {'confidence': 85,
              'datetime': '06-03-2021, 05:06:20',
              'surface_temperature_celcius': 59}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 88,
              'datetime': '07-03-2021, 04:16:10',
              'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 23,
 'hotspots': {'confidence': 86,
              'datetime': '09-03-2021, 13:23:40',
              'surface_temperature_celcius': 41}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 100,
              'datetime': '10-03-2021, 04:48:40',
              'surface_temperature_celcius': 105}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 100,
              'datetime': '10-03-2021, 04:46:20',
              'surface_temperature_celcius': 109}}
{'air_temper

              'datetime': '13-04-2021, 04:33:40',
              'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 91,
              'datetime': '13-04-2021, 04:33:30',
              'surface_temperature_celcius': 69}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 99,
              'datetime': '13-04-2021, 04:33:30',
              'surface_temperature_celcius': 85}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 90,
              'datetime': '13-04-2021, 04:33:30',
              'surface_temperature_celcius': 66}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 100,
              'datetime': '13-04-2021, 04:33:20',
              'surface_temperature_celcius': 96}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 89,
              'datetime': '13-04-2021, 04:33:20',
              'surface_temperature_celcius': 66}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 97,
              '

{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 95,
              'datetime': '18-04-2021, 00:24:50',
              'surface_temperature_celcius': 77}}
{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 83,
              'datetime': '18-04-2021, 00:24:40',
              'surface_temperature_celcius': 56}}
{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 83,
              'datetime': '18-04-2021, 00:24:40',
              'surface_temperature_celcius': 57}}
{'air_temperature_celcius': 22,
 'hotspots': {'confidence': 80,
              'datetime': '19-04-2021, 13:23:10',
              'surface_temperature_celcius': 39}}
{'air_temperature_celcius': 22,
 'hotspots': {'confidence': 83,
              'datetime': '19-04-2021, 13:18:20',
              'surface_temperature_celcius': 40}}
{'air_temperature_celcius': 22,
 'hotspots': {'confidence': 81,
              'datetime': '19-04-2021, 13:15:10',
              'surface_temperature_celcius': 39}}
{'air_temperatur

{'air_temperature_celcius': 5,
 'hotspots': {'confidence': 81,
              'datetime': '30-06-2021, 00:27:19',
              'surface_temperature_celcius': 48}}
{'air_temperature_celcius': 5,
 'hotspots': {'confidence': 81,
              'datetime': '30-06-2021, 00:27:19',
              'surface_temperature_celcius': 48}}
{'air_temperature_celcius': 6,
 'hotspots': {'confidence': 84,
              'datetime': '02-07-2021, 04:29:18',
              'surface_temperature_celcius': 52}}
{'air_temperature_celcius': 6,
 'hotspots': {'confidence': 84,
              'datetime': '02-07-2021, 04:29:18',
              'surface_temperature_celcius': 52}}
{'air_temperature_celcius': 6,
 'hotspots': {'confidence': 88,
              'datetime': '02-07-2021, 04:29:00',
              'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 6,
 'hotspots': {'confidence': 88,
              'datetime': '02-07-2021, 04:29:00',
              'surface_temperature_celcius': 58}}
{'air_temperature_celc

#### e) Find the top 10 records with the highest surface temperature (°C).

In [15]:
cursor = db.embedded_climate_hotspot.aggregate([
    # since necessary detail (i.e., surface temperature) is in an associative array, hence I need to use unwind
    {"$unwind":"$hotspots"},
    # make sure the object id is not printed out
    {"$project":{"_id":0,"hotspots._id":0}},
    # sort the documents decending by its surface temperature
    {"$sort" : {"hotspots.surface_temperature_celcius":-1}},
    # only shows the top 10 documents
    {"$limit" : 10 }
])

"""
How to interpret my output:
My output shows all the details for the top 10 records with the highest surface temperature. 
The reason for this is because this is more convenient as it allows the client/user to retrieve all 
the details for these records. 
"""

for document in cursor: 
    document['date'] = document['date'].strftime('%d-%m-%Y') # make the date more readable
    document['hotspots']['date'] = document['hotspots']['date'].strftime('%d-%m-%Y') # make the date more readable
    # make the datetime more readable
    document['hotspots']['datetime'] = document['hotspots']['datetime'].strftime("%d-%m-%Y, %H:%M:%S")
    pprint(document) 

{'GHI_w/m2': 122,
 'air_temperature_celcius': 15,
 'date': '18-04-2021',
 'hotspots': {'confidence': 100,
              'date': '18-04-2021',
              'datetime': '18-04-2021, 04:52:00',
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celcius': 124},
 'max_wind_speed': 9.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'GHI_w/m2': 140,
 'air_temperature_celcius': 16,
 'date': '04-04-2021',
 'hotspots': {'confidence': 100,
              'date': '04-04-2021',
              'datetime': '04-04-2021, 04:32:50',
              'latitude': -36.343,
              'longitude': 142.1986,
              'surface_temperature_celcius': 123},
 'max_wind_speed': 12.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 47.5,
 'station': 948701,
 'windspeed_knots': 5.4}
{'GHI_w/m2': 121,
 'air_temperature_celcius': 14,
 'date': '01-05-2021',
 'hotspots': {'confidence': 100,
           

#### f) Find the number of fires each day. You are required to only display the total number of fires and the date in the output.

In [16]:
cursor = db.embedded_climate_hotspot.aggregate([
    {"$project":{"date":1,"number_of_fires": {"$size":"$hotspots"},"_id":0}},
    {"$sort" : {"date":1}} # not required to sort the date but just to make the output look neat
])

for document in cursor: 
    document['date'] = document['date'].strftime('%d-%m-%Y') # make the date more readable
    pprint(document) 

{'date': '31-12-2020', 'number_of_fires': 0}
{'date': '01-01-2021', 'number_of_fires': 0}
{'date': '02-01-2021', 'number_of_fires': 0}
{'date': '03-01-2021', 'number_of_fires': 0}
{'date': '04-01-2021', 'number_of_fires': 0}
{'date': '05-01-2021', 'number_of_fires': 0}
{'date': '06-01-2021', 'number_of_fires': 0}
{'date': '07-01-2021', 'number_of_fires': 0}
{'date': '08-01-2021', 'number_of_fires': 0}
{'date': '09-01-2021', 'number_of_fires': 0}
{'date': '10-01-2021', 'number_of_fires': 0}
{'date': '11-01-2021', 'number_of_fires': 0}
{'date': '12-01-2021', 'number_of_fires': 0}
{'date': '13-01-2021', 'number_of_fires': 0}
{'date': '14-01-2021', 'number_of_fires': 0}
{'date': '15-01-2021', 'number_of_fires': 0}
{'date': '16-01-2021', 'number_of_fires': 0}
{'date': '17-01-2021', 'number_of_fires': 0}
{'date': '18-01-2021', 'number_of_fires': 0}
{'date': '19-01-2021', 'number_of_fires': 0}
{'date': '20-01-2021', 'number_of_fires': 0}
{'date': '21-01-2021', 'number_of_fires': 0}
{'date': '

#### g) Find the records of fires where the confidence is below 70

In [17]:
cursor = db.embedded_climate_hotspot.aggregate([
    # since necessary detail is in an associative array, hence I need to use unwind
    {"$unwind":"$hotspots"},
    # only include the documents when its confidence is below 70
    {"$match": {"hotspots.confidence":{"$lt":70}}},
    # make sure object id is not included
    {"$project":{"_id":0,"hotspots._id":0}}
])

"""
How to interpret my output:
Similar to question e, my output shows all the details for all the documents when its confidence is below 70. 
The reason for this is because this is more convenient as it allows the client/user to retrieve all 
the details for these records. 
"""

for document in cursor: 
    document['date'] = document['date'].strftime('%d-%m-%Y') # make the date more readable
    document['hotspots']['date'] = document['hotspots']['date'].strftime('%d-%m-%Y') # make the date more readable
    # make the datetime more readable
    document['hotspots']['datetime'] = document['hotspots']['datetime'].strftime("%d-%m-%Y, %H:%M:%S")
    pprint(document) 

{'GHI_w/m2': 177,
 'air_temperature_celcius': 21,
 'date': '08-03-2021',
 'hotspots': {'confidence': 68,
              'date': '08-03-2021',
              'datetime': '08-03-2021, 04:51:00',
              'latitude': -37.7885,
              'longitude': 141.9352,
              'surface_temperature_celcius': 55},
 'max_wind_speed': 13.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 51.7,
 'station': 948701,
 'windspeed_knots': 7.2}
{'GHI_w/m2': 192,
 'air_temperature_celcius': 23,
 'date': '09-03-2021',
 'hotspots': {'confidence': 54,
              'date': '09-03-2021',
              'datetime': '09-03-2021, 03:57:00',
              'latitude': -37.7171,
              'longitude': 147.5866,
              'surface_temperature_celcius': 44},
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 53.4,
 'station': 948701,
 'windspeed_knots': 6.1}
{'GHI_w/m2': 149,
 'air_temperature_celcius': 19,
 'date': '10-03-2021',
 'hotspots': {'confidence': 55,
             

 'station': 948701,
 'windspeed_knots': 13.2}
{'GHI_w/m2': 185,
 'air_temperature_celcius': 21,
 'date': '07-04-2021',
 'hotspots': {'confidence': 68,
              'date': '07-04-2021',
              'datetime': '07-04-2021, 12:50:40',
              'latitude': -37.3344,
              'longitude': 149.3933,
              'surface_temperature_celcius': 47},
 'max_wind_speed': 20.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 47.0,
 'station': 948701,
 'windspeed_knots': 13.2}
{'GHI_w/m2': 185,
 'air_temperature_celcius': 21,
 'date': '07-04-2021',
 'hotspots': {'confidence': 61,
              'date': '07-04-2021',
              'datetime': '07-04-2021, 12:50:40',
              'latitude': -37.3511,
              'longitude': 149.4088,
              'surface_temperature_celcius': 46},
 'max_wind_speed': 20.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 47.0,
 'station': 948701,
 'windspeed_knots': 13.2}
{'GHI_w/m2': 185,
 'air_temperature_celcius': 21,
 'date': '07-04-2021

 'max_wind_speed': 9.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 54.9,
 'station': 948701,
 'windspeed_knots': 5.9}
{'GHI_w/m2': 132,
 'air_temperature_celcius': 16,
 'date': '13-04-2021',
 'hotspots': {'confidence': 67,
              'date': '13-04-2021',
              'datetime': '13-04-2021, 04:26:30',
              'latitude': -37.7862,
              'longitude': 142.9823,
              'surface_temperature_celcius': 43},
 'max_wind_speed': 9.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 54.9,
 'station': 948701,
 'windspeed_knots': 5.9}
{'GHI_w/m2': 132,
 'air_temperature_celcius': 16,
 'date': '13-04-2021',
 'hotspots': {'confidence': 61,
              'date': '13-04-2021',
              'datetime': '13-04-2021, 04:26:30',
              'latitude': -37.7416,
              'longitude': 143.2885,
              'surface_temperature_celcius': 41},
 'max_wind_speed': 9.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 54.9,
 'station': 948701,
 'windspeed_knots':

              'longitude': 143.5099,
              'surface_temperature_celcius': 42},
 'max_wind_speed': 9.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'GHI_w/m2': 122,
 'air_temperature_celcius': 15,
 'date': '18-04-2021',
 'hotspots': {'confidence': 68,
              'date': '18-04-2021',
              'datetime': '18-04-2021, 04:44:50',
              'latitude': -37.9255,
              'longitude': 143.1236,
              'surface_temperature_celcius': 44},
 'max_wind_speed': 9.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'GHI_w/m2': 122,
 'air_temperature_celcius': 15,
 'date': '18-04-2021',
 'hotspots': {'confidence': 63,
              'date': '18-04-2021',
              'datetime': '18-04-2021, 04:44:50',
              'latitude': -37.8857,
              'longitude': 143.2777,
              'surface_temperature_celcius': 41},
 'max_wind_speed': 9.9,
 'precipi

 'date': '04-05-2021',
 'hotspots': {'confidence': 66,
              'date': '04-05-2021',
              'datetime': '04-05-2021, 04:44:40',
              'latitude': -36.3168,
              'longitude': 141.1422,
              'surface_temperature_celcius': 43},
 'max_wind_speed': 7.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 43.3,
 'station': 948701,
 'windspeed_knots': 2.6}
{'GHI_w/m2': 91,
 'air_temperature_celcius': 10,
 'date': '04-05-2021',
 'hotspots': {'confidence': 54,
              'date': '04-05-2021',
              'datetime': '04-05-2021, 04:44:40',
              'latitude': -36.3278,
              'longitude': 141.082,
              'surface_temperature_celcius': 39},
 'max_wind_speed': 7.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 43.3,
 'station': 948701,
 'windspeed_knots': 2.6}
{'GHI_w/m2': 91,
 'air_temperature_celcius': 10,
 'date': '04-05-2021',
 'hotspots': {'confidence': 66,
              'date': '04-05-2021',
              'datetime': '04-05

              'longitude': 143.7445,
              'surface_temperature_celcius': 40},
 'max_wind_speed': 8.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 44.0,
 'station': 948701,
 'windspeed_knots': 2.8}
{'GHI_w/m2': 81,
 'air_temperature_celcius': 9,
 'date': '13-05-2021',
 'hotspots': {'confidence': 55,
              'date': '13-05-2021',
              'datetime': '13-05-2021, 04:38:40',
              'latitude': -36.591,
              'longitude': 141.7194,
              'surface_temperature_celcius': 40},
 'max_wind_speed': 8.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 44.0,
 'station': 948701,
 'windspeed_knots': 2.8}
{'GHI_w/m2': 81,
 'air_temperature_celcius': 9,
 'date': '13-05-2021',
 'hotspots': {'confidence': 68,
              'date': '13-05-2021',
              'datetime': '13-05-2021, 04:38:40',
              'latitude': -36.8377,
              'longitude': 142.642,
              'surface_temperature_celcius': 44},
 'max_wind_speed': 8.0,
 'precipitation

 'station': 948702,
 'windspeed_knots': 14.4}
{'GHI_w/m2': 126,
 'air_temperature_celcius': 14,
 'date': '24-09-2021',
 'hotspots': {'confidence': 58,
              'date': '24-09-2021',
              'datetime': '24-09-2021, 04:04:19',
              'latitude': -37.397,
              'longitude': 148.121,
              'surface_temperature_celcius': 58},
 'max_wind_speed': 23.9,
 'precipitation ': ' 0.16G',
 'relative_humidity': 44.0,
 'station': 948702,
 'windspeed_knots': 14.4}
{'GHI_w/m2': 126,
 'air_temperature_celcius': 14,
 'date': '24-09-2021',
 'hotspots': {'confidence': 68,
              'date': '24-09-2021',
              'datetime': '24-09-2021, 04:04:18',
              'latitude': -37.452,
              'longitude': 148.115,
              'surface_temperature_celcius': 77},
 'max_wind_speed': 23.9,
 'precipitation ': ' 0.16G',
 'relative_humidity': 44.0,
 'station': 948702,
 'windspeed_knots': 14.4}
{'GHI_w/m2': 126,
 'air_temperature_celcius': 14,
 'date': '24-09-2021',
 

#### h) Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.

In [18]:
cursor = db.embedded_climate_hotspot.aggregate([
    # set preserveNullAndEmptyArrays to make sure climate data with no hotspot also will be included after unwind
    {"$unwind": {
        "path": "$hotspots",
        "preserveNullAndEmptyArrays": True
    }},
    # find the average surface temperature
    {"$group":{"_id":"$date","average":{"$avg":"$hotspots.surface_temperature_celcius"}}},
    # project necessary details and also round the average value to 2 decimal places
    {"$project":{"rounded_average_surface_temperature":{"$round":["$average",2]},"date":"$_id","_id":0}},
    {"$sort" : {"date":1}} # not required to sort the date but just to make the output look neat
])

"""
How to interpret my output:
If there is no hotspot for a particular date, the value for the rounded_average_surface_temperature will be None. Otherwise,
the value for this column will be average surface temperature on that date. Please note that I had rounded the average value
for better visualization. I sorted them based on date for better visualization.
"""

for document in cursor: 
    document['date'] = document['date'].strftime('%d-%m-%Y') # make the date more readable
    pprint(document) 

{'date': '31-12-2020', 'rounded_average_surface_temperature': None}
{'date': '01-01-2021', 'rounded_average_surface_temperature': None}
{'date': '02-01-2021', 'rounded_average_surface_temperature': None}
{'date': '03-01-2021', 'rounded_average_surface_temperature': None}
{'date': '04-01-2021', 'rounded_average_surface_temperature': None}
{'date': '05-01-2021', 'rounded_average_surface_temperature': None}
{'date': '06-01-2021', 'rounded_average_surface_temperature': None}
{'date': '07-01-2021', 'rounded_average_surface_temperature': None}
{'date': '08-01-2021', 'rounded_average_surface_temperature': None}
{'date': '09-01-2021', 'rounded_average_surface_temperature': None}
{'date': '10-01-2021', 'rounded_average_surface_temperature': None}
{'date': '11-01-2021', 'rounded_average_surface_temperature': None}
{'date': '12-01-2021', 'rounded_average_surface_temperature': None}
{'date': '13-01-2021', 'rounded_average_surface_temperature': None}
{'date': '14-01-2021', 'rounded_average_surface_

#### i) Find the top 10 records with the lowest GHI.

In [19]:
cursor = db.embedded_climate_hotspot.aggregate([
    {"$project":{"_id":0,"hotspots":0}},
    {"$sort" : {"GHI_w/m2":1}},
    {"$limit" : 10 }    # only the top 10 records with the lowest GHI
])

"""
How to interpret my output:
Similar to question e, my output shows all the details (i.e., climate data) for the top 10 records with the lowest GHI. 
The reason for this is because this is more convenient as it allows the client/user to retrieve all 
the details for these records. Besides that, since the question did not explicitly ask us to show the 
hotspots and to simplify the output, I did not show the hotspots. Otherwise, the output would be too long and complex.
If the marker want to see the hotspots data, please remove the "hotspots":0 above, thank you.
"""

for document in cursor: 
    document['date'] = document['date'].strftime('%d-%m-%Y') # make the date more readable
    pprint(document) 

{'GHI_w/m2': 47,
 'air_temperature_celcius': 5,
 'date': '02-08-2021',
 'max_wind_speed': 5.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 'air_temperature_celcius': 5,
 'date': '30-06-2021',
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00A',
 'relative_humidity': 34.9,
 'station': 948701,
 'windspeed_knots': 4.3}
{'GHI_w/m2': 49,
 'air_temperature_celcius': 5,
 'date': '01-07-2021',
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00I',
 'relative_humidity': 33.1,
 'station': 948701,
 'windspeed_knots': 5.8}
{'GHI_w/m2': 56,
 'air_temperature_celcius': 6,
 'date': '11-07-2021',
 'max_wind_speed': 5.1,
 'precipitation ': ' 0.01G',
 'relative_humidity': 39.0,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 57,
 'air_temperature_celcius': 6,
 'date': '15-07-2021',
 'max_wind_speed': 9.9,
 'precipitation ': ' 0.00I',
 'relative_humidity': 37.1,
 'station': 948701,
 'windspeed_knots': 5.8}
{'GHI_w/m2': 

#### j) Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.

In [20]:
cursor = db.embedded_climate_hotspot.aggregate([
    # find the records such that the precipitation column end with either G, D or F
    {"$match":{"precipitation ": {"$regex": "(G|D|F)$"}}}, 
    {"$project":{"_id":0,"hotspots":0}}
])

"""
How to interpret my output:
Since the question did not explicitly ask us to show the hotspots and to simplify the output, I did not show the hotspots. 
Hence, I only shows the climate data for the records with a 24-hour precipitation recorded between 0.20 to 0.35.
If the marker want to see the hotspots data, please remove the "hotspots":0 above, thank you.
"""

for document in cursor: 
    document['date'] = document['date'].strftime('%d-%m-%Y') # make the date more readable
    current = document['precipitation ']
    current = float(current[1:5])                            # get the amount of precipitation
    if current >= 0.20 and current <= 0.35:
        pprint(document) 

{'GHI_w/m2': 157,
 'air_temperature_celcius': 19,
 'date': '13-01-2021',
 'max_wind_speed': 18.1,
 'precipitation ': ' 0.31G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 'air_temperature_celcius': 17,
 'date': '29-03-2021',
 'max_wind_speed': 21.0,
 'precipitation ': ' 0.24G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 'air_temperature_celcius': 20,
 'date': '20-04-2021',
 'max_wind_speed': 15.9,
 'precipitation ': ' 0.31G',
 'relative_humidity': 53.5,
 'station': 948701,
 'windspeed_knots': 7.2}
{'GHI_w/m2': 102,
 'air_temperature_celcius': 11,
 'date': '26-04-2021',
 'max_wind_speed': 20.0,
 'precipitation ': ' 0.24G',
 'relative_humidity': 40.8,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 122,
 'air_temperature_celcius': 14,
 'date': '27-05-2021',
 'max_wind_speed': 19.0,
 'precipitation ': ' 0.35G',
 'relative_humidity': 48.1,
 'station': 948701,
 'windspeed_knots': 13